In [195]:
def eliminate_implication(exp):
    new_exp = exp
    while "->" in new_exp:
        parts = exp.split("->")
        if len(parts) == 2:
            new_exp = f"~({parts[0].strip()}) | ({parts[1].strip()})"


    return new_exp

def move_negation_inside(exp):
    new_exp = exp
    if exp.startswith("~(") and exp.endswith(")"):
        new_exp =  exp[2:-1]
        op_index = None
        for i in range(len(new_exp)):
            if new_exp[i] in ["&", "|", "->"]:
                op_index = i
                break
        if op_index is not None:
            if new_exp[op_index] == "&":
                return f"(~{new_exp[:op_index].strip()} | ~{new_exp[op_index+1:].strip()})"
            elif new_exp[op_index] == "|":
                return f"(~{new_exp[:op_index].strip()} & ~{new_exp[op_index+1:].strip()})"

    # Check if the modified expression contains "¬∀"
    while "~∀" in new_exp:
        parts = new_exp.split("~∀", 1)
        if len(parts) == 2:
            before_forall, after_forall = parts
            after_forall = after_forall.strip()
            after_forall = after_forall[0] + " ~" + after_forall[1:]

            new_exp = before_forall + "∃" + after_forall


    # Check if "¬∃" is in the expression
    while "~∃" in new_exp:
        parts = new_exp.split("~∃", 1)
        if len(parts) >= 2:
            before_exists, after_exists = parts
            after_exists = after_exists.strip()
            after_exists = after_exists[0] + " ~" + after_exists[1:]
            new_exp = before_exists + "∀" + after_exists


    return new_exp


def remove_double_not(exp):
    while "~~" in exp:
        exp = exp.replace("~~", "")

    return exp



def standardize_variable_scope(exp):

    var_map = []
    var_count = 0
    new_string = ""
    def new_var(var_map):
        nonlocal var_count
        var_count += 1
        next_letter_ascii = ord('x') + var_count
        next_letter_ascii = (next_letter_ascii - ord('a')) % 26 + ord('a')  # Loop back to 'a' if exceeding 'z'
        next_letter = chr(next_letter_ascii)
        if next_letter in var_map:
            return new_var(var_map)
        return next_letter


    def replace_char(string, char_to_replace, replacement_char, start_index):
        index = string.find(char_to_replace, start_index, len(exp))  # Search for the character within the next 5 characters
        if index != -1:  # If the character is found within the specified range
        # Replace the character at the found index with the replacement character
            nonlocal new_string
            new_string = string[:index] + replacement_char + string[index + 1:]
            return new_string  # Return the modified string
        else:
            return string


    i = 0
    while i < len(exp) - 1:
        if exp[i] == '∀' or exp[i] == '∃':
            quantifier = exp[i]
            if exp[i + 1].isalpha():
                var = exp[i + 1]
                if var in var_map:
                    # If the variable is seen again after another quantifier, replace it with a new variable
                    new_variable = new_var(var_map=var_map)
                    exp = exp[:i + 1] + new_variable + exp[i + 2:]
                    exp = replace_char(exp, var, new_variable, i + 1)
                    var_map.append(new_variable)
                else:
                    var_map.append(var)
            i += 2
        else:
            i += 1
    return exp




def prenex_form(exp):
    def is_quantifier(char):
        return char == '∀' or char == '∃'


    for i in range (len(exp)):
        if is_quantifier(exp[i]):
            quantifier = exp[i]
            var = exp[i+1]
            exp = quantifier  + var + exp[:i] + exp[i+2:]

    return exp


skolem_temp = 97 # letter a
def replace_from_index(whole_string, changed, replacement, start):
    prefix = whole_string[:start]
    rest = whole_string[start:]
    replaced_string = rest.replace(changed, replacement)
    return replaced_string
def skolemization(exp):
    global skolem_temp
    temp = exp
    for i in range(len(exp)):
        if(i<len(exp) and exp[i]=='∃'):
            to_change = temp[i+1]
            skolem_fun = 'F(' + chr(skolem_temp) + ')'
            skolem_temp+=1
            temp = temp.replace(temp[i:i+2], '')
            temp = replace_from_index(temp, to_change, skolem_fun, i)
    return temp


def eliminate_universal_quantifiers(exp):

    def is_uni_quantifier(char):
        return char == '∀'

    def remove_chars(string, char):
        index = string.find(char)
        if index != -2:
            return string[:index] + string[index + 2:]
        return string

    i = 0
    while i < len(exp):
        if is_uni_quantifier(exp[i]):
            quantifier = exp[i]
            exp = remove_chars(exp, quantifier)
        else:
            i += 1

    return exp


def convert_to_cnf(exp):
    exp = eliminate_implication(exp)
    exp = move_negation_inside(exp)
    exp = remove_double_not(exp)
    exp = standardize_variable_scope(exp)
    exp = prenex_form(exp)
    exp = skolemization(exp)
    exp = eliminate_universal_quantifiers(exp)
    if(exp[0]=='(' and exp[-1]==')'):
        exp = exp[1:-1]
    return exp


def clause_conversion(exp):
    exp = exp.split("&")
    for i in range(len(exp)):
        exp[i] = exp[i].strip()
    return exp


rename_temp = 97 # letter a
def rename_variables(exp):
    global rename_temp
    res = []
    for el in exp:
        result = ""
        for char in el:
            if char.islower():
                result += chr(rename_temp)
                rename_temp +=1
            else:
                result += char
        res.append(result)
    for i in range(len(res)):
        res[i] = res[i].split(" | ")
    return res


def resolve(clause1, clause2):
    resolvent = set()
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1 == '~' + literal2 or literal2 == '~' + literal1:
                # union for set / adding to set
                resolvent |= set(clause1 + clause2) - {literal1, literal2}
    return list(resolvent)
def resolution(exp1, exp2):
    for clause1 in exp1:
        for clause2 in exp2:
            resolvent = resolve(clause1, clause2)
            if resolvent == []:
                return False
    return True

def solve(exp1, exp2):
    print("Original1:", exp1)
    print("Original2:", exp2)
    print("")
    exp1 = convert_to_cnf(exp1)
    exp2 = convert_to_cnf(exp2)
    print("CNF1:", exp1)
    print("CNF2:", exp2)
    print("")
    exp1 = clause_conversion(exp1)
    exp2 = clause_conversion(exp2)
    print("Clause1:", exp1)
    print("Clause2:", exp2)
    print("")
    exp1 = rename_variables(exp1)
    exp2 = rename_variables(exp2)
    print("Final1:", exp1)
    print("Final2:", exp2)
    print("")
    consistent = resolution(exp1, exp2)
    print("Consistent" if consistent else "Inconsistent")

In [196]:
solve('∀x P(x) & (∃y Q(y) -> R(y))', '~∀x p & ~∀x q')

Original1: ∀x P(x) & (∃y Q(y) -> R(y))
Original2: ~∀x p & ~∀x q

CNF1: ~ P(x) | ~( Q(F(a))) | (R(F(a)))
CNF2:  p &  q

Clause1: ['~ P(x) | ~( Q(F(a))) | (R(F(a)))']
Clause2: ['p', 'q']

Final1: [['~ P(a)', '~( Q(F(b)))', '(R(F(c)))']]
Final2: [['d'], ['e']]

Inconsistent
